# RQ1

In [63]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

In [127]:
import pandas as pd
import os

def get_path_apgd(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv') and 'apfd' in file_absolute_path:
                    path_list.append(file_absolute_path)
    return path_list

def check_attack(path):
    attack_check_list = ['pgdattack', 'nodeembeddingattack', 'randomattack', 'dice', 'minmax']
    for i in attack_check_list:
        if i in path:
            return True
    return False


def check_weight_stacking_voting(s):
    if 'weight' in s:
        return 1
    if 'stacking' in s:
        return 1
    if 'voting' in s:
        return 1
    return 0

path_list = get_path_apgd('/Users/yinghua.li/Documents/Pycharm/NodeRank/res')
path_list = sorted([i for i in path_list if not check_attack(i)])
# path_list.append('/Users/yinghua.li/Documents/Pycharm/NodeRank/res/apfd_citeseer_gcn.csv')
path_list = sorted(path_list)
data_list = []
for path in path_list:
    df = pd.read_csv(path, header=None, names=['subject', 'apfd'])
    data_list.append(df)
df = pd.concat(data_list, ignore_index=True)
df['data'] = [i.split('_')[0] for i in df['subject']] 
df['gnn'] = [i.split('_')[1] for i in df['subject']] 
df['method'] = [i.split('_')[2] for i in df['subject']]

dic = {'gat': 'GAT', 'gcn': 'GCN', 'tagcn': 'TAGCN', 'graphsage': 'GraphSAGE', 'xgb': 'XGB', 'lgb': 'LGB', 'rf': 'RF', 
       'lr': 'LR', 'mutation': 'Mutation', 'fusion': 'NodeRank',
       'deepGini': 'DeepGini', 'leastConfidence': 'Least Confidence', 'margin': 'Margin', 'random': 'Random'}
dic_data = {'citeseer': 'CiteSeer', 'cora': 'Cora', 'lastfm': 'LastFM', 'pubmed': 'PubMed'}
df['data'] = [dic_data[i] for i in df['data']]

df['gnn'] = [dic[i] for i in df['gnn']]
df['method'] = [dic[i] for i in df['method']]
df = df[(df['method']!='XGB')&(df['method']!='LGB')&
        (df['method']!='RF')&(df['method']!='LR')&
        (df['method']!='Mutation')]

df['check'] = df['subject'].apply(check_weight_stacking_voting)
df = df[df['check']==0]
del df['check']

In [165]:
approach_list = ['Random', 'Margin','Least Confidence', 'DeepGini', 'NodeRank']
data_list = ['CiteSeer', 'Cora', 'LastFM', 'PubMed']
gnn_list = ['GAT', 'GCN', 'GraphSAGE', 'TAGCN']

res_list = []
for method in approach_list:
    tmp_list = [method]
    for data in data_list:
        for gnn in gnn_list:
            apfd = df[(df['method']==method) & (df['data']==data) & (df['gnn']==gnn)]['apfd'].values[0]
            tmp_list.append(apfd)
    res_list.append(tmp_list)  
    
df_re = pd.DataFrame(res_list, columns=['Method']+['GAT', 'GCN', 'GraphSAGE', 'TAGCN']*4)
df_re

,Method,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN,GAT,GCN,GraphSAGE,TAGCN
0,Random,0.518912,0.492917,0.493088,0.511665,0.503254,0.478755,0.517079,0.493682,0.500594,0.504364,0.524906,0.500270,0.501879,0.491635,0.492895,0.507982
1,Margin,0.681322,0.672569,0.682115,0.680334,0.743866,0.728317,0.718312,0.734541,0.653423,0.750178,0.772097,0.763214,0.648763,0.599527,0.659048,0.661974
2,Least Confidence,0.675313,0.672070,0.665940,0.672819,0.752761,0.744302,0.735226,0.741384,0.638566,0.737419,0.780938,0.772234,0.662187,0.709032,0.658542,0.658466
3,DeepGini,0.638040,0.643868,0.611970,0.627684,0.720020,0.726827,0.710136,0.693224,0.585646,0.691724,0.759221,0.739889,0.637740,0.693837,0.608250,0.615310
4,NodeRank,0.717976,0.700982,0.729987,0.722619,0.821919,0.819389,0.819161,0.792463,0.824753,0.830142,0.830645,0.822687,0.790333,0.774411,0.816510,0.811550


In [162]:
len(cols)

80

In [106]:
4*4

16

In [154]:
set(df['gnn'])

{'GAT', 'GCN', 'GraphSAGE', 'TAGCN'}